<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект_2: Анализ резюме из HeadHunter

In [761]:
import pandas as pd
import numpy as np
import matplotlib as pl
import psycopg2


DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

connection = psycopg2.connect(
    dbname = DBNAME,
    user = USER,
    password = PASSWORD,
    host = HOST,
    port = PORT
)

def print_query(query_i):
    df = pd.read_sql_query(query_i, connection)
    print(df)


# Предварительный анализ данных

1. Сколько вакансий есть в базе


In [762]:
query_3_1 = f'''select
            count(vacancies.id)
            from vacancies
            '''

print_query(query_3_1)


C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   count
0  49197


База ваканций насчитывает 49197 вакансию

2. Сколько работадателей есть в базе

In [763]:
query_3_2 = f'''select
            count(employers.id)
            from employers
            '''
print_query(query_3_2)

   count
0  23501


C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


В базе назодятся 23501 работадатель

3. Сколько регионов есть в базе

In [764]:
query_3_3 = f'''select
            count(areas.id)
            from areas
            '''
print_query(query_3_3)

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   count
0   1362


База насчитывает 1362 региона

4. Сколько сфер деятельности есть в базе

In [765]:
query_3_4 = f'''select
            count(industries.id)
            from industries
            '''
print_query(query_3_4)

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   count
0    294


294 сферы деятельности указаны в базе данных

Общий вывод. Достаточна обьемная база данных для построения выполнения запросов извлечения из нее необходимой информации и составляние выводов по тем или иным вопросам.

# Детальный анализ вакансий

1. Пятерка лидеров регионов по количеству вакансий

In [766]:
query_4_1 = f'''with temp as(select 
            count(vacancies.id) as cnt,
            areas.name
            from vacancies
            join areas on areas.id = vacancies.area_id
            group by areas.name
            order by 1 desc
            limit 5)
            
            select 
                temp.name,
                temp.cnt
            from temp
                union all 
            select 
                'total of 5',
                sum(temp.cnt) as sum_1
            from temp
                union all
            select 
                'total vacancies',
                count(vacancies.id)
            from vacancies
           '''

print_query(query_4_1)
   

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


              name      cnt
0           Москва   5333.0
1  Санкт-Петербург   2851.0
2            Минск   2112.0
3      Новосибирск   2006.0
4           Алматы   1892.0
5       total of 5  14194.0
6  total vacancies  49197.0


Москва, Санкт Петербург, Минск, Новосибирск и Алматы являются лидерами по количеству прелагаемых вакансий. Эти пять городов предлагают чуть больше 14 тысяч позиций и имеют треть от всех вакансии на рынке 

2.  У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

In [767]:
query_4_2 = f'''select 
            count(id)
            from vacancies
            where salary_from > 0 or salary_to > 0
            '''
print_query(query_4_2)

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   count
0  24073


в 24073 вакансиях присутствуют сведения о зарплате 

3. Средние значения для нижней и верхней границы зарплатной вилки

In [768]:
query_4_3 = f'''select
                round(avg(salary_from)) mean_salary_from
                from vacancies
                where salary_from is not Null
                union 
                select
                round(avg(salary_to)) 
                from vacancies
                where salary_to is not null
            ''' 
print_query(query_4_3)

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   mean_salary_from
0           71065.0
1          110537.0


Среднии значения для нижней и верхней зарплатной вилки варьируются от 71.075 до 110.537 руб.  

4. Количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства. Какая пара находится на втором месте по популярности?

In [769]:
query_4_4 = f'''select
                count(schedule),
                vacancies.schedule,
                count(employment),
                vacancies.employment
                from vacancies
                group by vacancies.schedule, vacancies.employment
                order by 1 desc
            '''
print_query(query_4_4)

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


    count          schedule           employment
0   35367       Полный день     Полная занятость
1    7802  Удаленная работа     Полная занятость
2    1593     Гибкий график     Полная занятость
3    1312  Удаленная работа  Частичная занятость
4     940    Сменный график     Полная занятость
5     569       Полный день           Стажировка
6     367    Вахтовый метод     Полная занятость
7     347       Полный день  Частичная занятость
8     312     Гибкий график  Частичная занятость
9     141       Полный день     Проектная работа
10    133  Удаленная работа     Проектная работа
11    116     Гибкий график           Стажировка
12    101    Сменный график  Частичная занятость
13     64  Удаленная работа           Стажировка
14     18     Гибкий график     Проектная работа
15     12    Сменный график           Стажировка
16      2    Вахтовый метод     Проектная работа
17      1    Сменный график     Проектная работа


Полная занятость с полным рабочим денем является самой распространенным видом в предлагаемых позициях

5. Требуемый опыт работы в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [770]:
query_4_5 = f'''select 
                count(experience),
                vacancies.experience
                from vacancies
                group by vacancies.experience
                having vacancies.experience is not null
                order by 1 asc
            '''
print_query(query_4_5)

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   count          experience
0   1337         Более 6 лет
1   7197           Нет опыта
2  14511       От 3 до 6 лет
3  26152  От 1 года до 3 лет


Для 26.152 вакансий ищут соискателей с опытом работы от 1 года до 3 лет и 14.511 с опытом от 3 до 6 лет. Спросом пользуются молодые специалисты. Большинство вакансии для специалистов с опытом до 6 лет

# Анализ работодателей

1. Какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [771]:
query_5_1 = f'''
            select 
            count(vacancies.employer_id),
            employers.name
            from employers
            join vacancies on employers.id = vacancies.employer_id
            group by vacancies.employer_id, employers.name
            order by 1 desc
            limit 5 
            '''        
print_query(query_5_1)                        

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   count           name
0   1933         Яндекс
1    491     Ростелеком
2    444       Тинькофф
3    428           СБЕР
4    331  Газпром нефть


Яндекс предлагает соискателем почти 2 тысячи вакансий, является работадателем с самым большим предложением на рабочем рынке. Яндекс предлагает в 4 раза больше вакансий чем Ростелеком который находится на втором месте в списке по количеству вакансий с 491 открытыми позициями и больше чем 4 последуйщих работадателей вместе взятых после Яндекса

2. Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. 

In [772]:
query_5_2  = f'''
            select
                areas.name,
                count(distinct employers.id) as num_employers,
                count(distinct vacancies.id) as num_vacancies
            from 
                areas
            left join        
                employers on employers.area = areas.id
            left join
                vacancies on vacancies.area_id = areas.id
            where vacancies.id is null
            group by areas.id
            order by num_employers desc
            '''
            
print_query(query_5_2) 

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


                   name  num_employers  num_vacancies
0                Россия            410              0
1             Казахстан            207              0
2    Московская область             75              0
3    Краснодарский край             19              0
4    Ростовская область             18              0
..                  ...            ...            ...
588             Скидель              0              0
589          Белоозерск              0              0
590           Орловский              0              0
591          Павловская              0              0
592             Таксимо              0              0

[593 rows x 3 columns]


Россия занимает первое место в котором наибольшее количество работадателей но нет вакансии. На третьем месте московская область. Возможно что данные немного искажены так как московская область находится в россии и регион "Россия" уже включает в себя данные с других областей и регионов страны. 

3. Количество регионов для каждого работодателя, в которых он публикует свои вакансии и максимальное значение

In [773]:
query_5_3 = f'''
            select 
                count(distinct area_id),
                employers.name
            from vacancies 
            join employers on vacancies.employer_id = employers.id
            group by employers.name, employers.area
            --having count(distinct area_id) = 1
            order by 1 desc
            '''
print_query(query_5_3) 

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


       count                                              name
0        181                                            Яндекс
1        152                                        Ростелеком
2        116                                        Спецремонт
3         88                            Поляков Денис Иванович
4         71                                         ООО ЕФИН 
...      ...                                               ...
14897      1                                         TheRabota
14898      1  The Skolkovo Institute of Science and Technology
14899      1                                         Theyr Ltd
14900      1                                           THINK24
14901      1                                        think-cell

[14902 rows x 2 columns]


Топ три работадателя (Яндекс, Ростелеком, Спецремонт) размещают вакансии больше чем в 100 разных регионов каждый. В 12.482 регионов есть по одной всего лишь вакансии. 

4. Количество работадателей у которых не указана сфера деятельности

In [774]:
query_5_4 = f'''
            select
            count(e.name)
            from employers e
            left join employers_industries ei on e.id = ei.employer_id
            left join industries i on i.id = ei.industry_id
            where i is null
            '''

print_query(query_5_4) 

   count
0   8419


C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


8419 работадателей не указали сферу деятельности

5. Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [775]:
query_5_5 = f'''
            select 
                e.name,
                count(ei.industry_id)
            from employers e
            join employers_industries as ei on e.id = ei.employer_id 
            group by e.id
            having count(industry_id) = 4 
            order by name asc
            limit 3
            '''
print_query(query_5_5) 

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


           name  count
0  101 Интернет      4
1      21vek.by      4
2          2ГИС      4


6.  У какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [776]:
query_5_6 = f'''
            select 
            count(e.id)
            from employers e
            join employers_industries as ei on e.id = ei.employer_id
            join industries i on i.id = ei.industry_id
            where i.name = 'Разработка программного обеспечения'
            group by e.id, ei.employer_id, ei.industry_id, i.id
            '''
print_query(query_5_6)

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


      count
0         1
1         1
2         1
3         1
4         1
...     ...
3548      1
3549      1
3550      1
3551      1
3552      1

[3553 rows x 1 columns]


 У 3553 работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

7. Для компании «Яндекс» выведите список регионов-миллионников , в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании, собранных в этой таблице. 

In [777]:
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')

cities_here = str(page.find('table', class_= "standard"))
cities_here = cities_here.split('title="')[3:]

million_cities = []
for i in cities_here:
    million_cities.append(i.split('">')[0])

# print(million_cities)
    
million_cities = tuple(million_cities)
    

query_5_7 = f'''
            with temp as( 
            select 
                a.name as name,
                count(v.id) as cnt
            from employers e
                join vacancies v on e.id = v.employer_id
                join areas a on a.id = v.area_id
            where e.name = 'Яндекс' 
            group by a.name
            having a.name in {million_cities})
            
            select 
                temp.name,
                temp.cnt
            from temp
                union all 
            select 
                'total',
                sum(temp.cnt)
            from temp
            '''
print_query(query_5_7)

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


               name    cnt
0         Волгоград   24.0
1           Воронеж   32.0
2      Екатеринбург   39.0
3            Казань   25.0
4         Краснодар   30.0
5        Красноярск   23.0
6            Москва   54.0
7   Нижний Новгород   36.0
8       Новосибирск   35.0
9              Омск   21.0
10            Пермь   25.0
11   Ростов-на-Дону   25.0
12           Самара   26.0
13  Санкт-Петербург   42.0
14              Уфа   26.0
15        Челябинск   22.0
16            total  485.0


Компания Яндекс прелагает 485 вакансий в 16 разных городах миллионниках. 

# Предметный анализ

1.Сколько вакансий имеет отношение к данным?

In [778]:
query_6_1 = f'''
                select 
                count(v.id)
                from vacancies v
                where v.name ilike '%data%' or v.name ilike '%данн%'
            '''
            
print_query(query_6_1) 

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   count
0   1771


 1771 вакансий иммет отношение к данным.

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста?

In [779]:
query_6_2 = f'''with temp as(
                    select
                    --count(v.name)
                    v.name,
                    v.experience,
                    v.employment
                    from vacancies v
                    where v.name ilike '%data scientist%' or 
                        v.name ilike '%data science%' or 
                        v.name ilike '%ML%' and v.name not ilike '%html%' or 
                        v.name ilike '%исследователь данных%' or
                        v.name ilike '%machine learning%' or 
                        v.name ilike '%машин%обучен%')
                
                select
                count(*)
                from temp t
                where t.name ilike '%junior%' or
                    t.experience ilike '%нет опыта%' or
                    t.employment ilike '%стажировка%' 
            '''
print_query(query_6_2)

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   count
0     51


51 вакансий  в базе данных могут подойти для начинающего дата-сайентиста 

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?


In [780]:
query_6_3  = f'''
            with temp as(
                    select
                    --count(v.name)
                    v.name,
                    v.experience,
                    v.employment,
                    v.key_skills
                    from vacancies v
                    where v.name ilike '%data scientist%' or 
                        v.name ilike '%data science%' or 
                        v.name ilike '%ML%' and v.name not ilike '%html%' or 
                        v.name ilike '%исследователь данных%' or
                        v.name ilike '%machine learning%' or 
                        v.name ilike '%машин%обучен%')
                        
            select*
            from temp
            where key_skills ilike '%sql%' or
                key_skills ilike '%postgres%'
            '''
            
print_query(query_6_3) 

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


                                               name          experience   
0                           Junior+ CV/ML developer  От 1 года до 3 лет  \
1                                    Data Scientist  От 1 года до 3 лет   
2                            QA Automation TeamLead       От 3 до 6 лет   
3                               PHP Senior/TeamLead  От 1 года до 3 лет   
4                                    Data Scientist  От 1 года до 3 лет   
..                                              ...                 ...   
224                     Senior Developer (teamlead)         Более 6 лет   
225   Руководитель группы разработки (TeamLead PHP)       От 3 до 6 лет   
226  Machine Learning and Computer vision developer  От 1 года до 3 лет   
227                   Middle+/Senior Data Scientist       От 3 до 6 лет   
228     Системный аналитик /Middle+/Senior/TeamLead       От 3 до 6 лет   

           employment                                         key_skills  
0    Полная занятость   

Для 229 вакансии на позицию Data Science соискателям нобзодимо знание SQL 

4. Насколько пайтон актуален в вакансиях? Количество вакансий, в которых в качестве ключевого навыка указан Python?

In [781]:

query_6_4  = f'''
            with temp as(
                    select
                    --count(v.name)
                    v.name,
                    v.experience,
                    v.employment,
                    v.key_skills
                    from vacancies v
                    where v.name ilike '%data scientist%' or 
                        v.name ilike '%data science%' or 
                        v.name ilike '%ML%' and v.name not ilike '%html%' or 
                        v.name ilike '%исследователь данных%' or
                        v.name ilike '%machine learning%' or 
                        v.name ilike '%машин%обучен%')
                        
            select
            count(*)
            from temp
            where key_skills ilike '%python%'
            '''
            
print_query(query_6_4) 

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   count
0    357


Для 357 вакансии на позицию Data Science соискателям нобзодимо знание Python 

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS? Ответ округлите до двух знаков после точки-разделителя.

In [782]:
query_6_5  = f'''
            with temp as(
                    select
                    --count(v.name)
                    v.name,
                    v.experience,
                    v.employment,
                    v.key_skills
                    from vacancies v
                    where v.name ilike '%data scientist%' or 
                        v.name ilike '%data science%' or 
                        v.name like '%ML%' and v.name not like '%html%' or 
                        v.name ilike '%исследователь данных%' or
                        v.name ilike '%machine learning%' or 
                        v.name ilike '%машин%обучен%')
                        
            select
                round(avg(length(key_skills)-length(replace(key_skills, CHR(9), ''))+1), 2)
            from temp
                where key_skills is not null
            '''
query_6_5_1  = f'''
                    select
                    v.key_skills
                    from vacancies v
                    where v.name ilike '%data scientist%' or 
                        v.name ilike '%data science%' or 
                        v.name like '%ML%' and v.name not like '%html%' or 
                        v.name ilike '%исследователь данных%' or
                        v.name ilike '%machine learning%' or 
                        v.name ilike '%машин%обучен%'
                    
            '''
            
print_query(query_6_5) 
print_query(query_6_5_1)

C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


   round
0   6.64


C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


                                            key_skills
0                        JavaScript\tSass\tCSS3\tHTML5
1            Python\tSQL\tРабота в команде\tOpenCV\tML
2    JavaScript\tCSS3\tHTML5\tjQuery\tКроссбраузерн...
3    Oracle Pl/SQL\tБазы данных\tMachine Learning\t...
4    Python\tOpenCV\tTensorflow\tComputer Vision\tН...
..                                                 ...
604  CSS3\tHTML5\tAdobe Photoshop\tGit\tКроссбраузе...
605  Twig\tDocker-compose\tGulp\tCMS Wordpress\tVue...
606               Ответственность, способность учиться
607  HTML5\tjQuery\tCSS3\tBootstrap\tJavaScript\tPH...
608               JavaScript\tHTML5\tjQuery\tCSS\tHTML

[609 rows x 1 columns]


В среднем в вакансиях для DS указывают 6 ключевых навыков? Из ключевых можно отметить: HTML, Python, SQL, JavaScript, HTML5, ML/Computer Vision

6.  Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [783]:
query_6_6 = f'''
                 select
                    v.experience,
                    round(avg(coalesce((v.salary_to + v.salary_from)/2, v.salary_from, v.salary_to)), 0) as avg_sal
                 from 
                    vacancies v
                where 
                    ((v.name ilike '%data scientist%')
                    or (v.name ilike '%data science%') 
                    or (v.name ilike '%исследователь данных%')
                    or ((v.name like '%ML%') and (v.name not ilike '%HTML%'))
                    or (v.name ilike '%machine learning%')
                    or (v.name ilike '%машин%обучен%'))
                    and ((v.salary_from is not null)or(v.salary_to is not null))
                group by 1
                '''
print_query(query_6_6) 


C:\Users\geras\AppData\Local\Temp\ipykernel_16720\4031417728.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_i, connection)


           experience   avg_sal
0           Нет опыта   74643.0
1  От 1 года до 3 лет  139675.0
2       От 3 до 6 лет  243115.0


В среднем дата сайентисты с опытом более трех лет могут расчитывать на зарплату 240 тысяч

В целом основываясь на выводах с запросов можно сделать заключение что профессия дата-сайентиста  является одной из самых востребованных и популярных на рынке труда. Для большинства вакансий необходим опыт от одного года и выше. При этом зарплаты дата саентистов сильно различаются в зависимости от опыта работы. В целом можно охарактеризовать эту профессию с высоким уровнем зароботной платы

In [784]:
connection.close()